In [1]:
using Tables
using MarkdownTables
using ProgressMeter
include("datasetloader.jl")
include("iso639code.jl")

normcode (generic function with 2 methods)

In [2]:
TLANGS = ["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", "eng", "epo", "pes", "fin",
 "fra", "hau", "srp", "heb", "hin", "hun", "ido", "ina", "isl", "ita", "jpn", "kab", "kor", "ckb", 
 "lat", "lit", "mar", "mkd", "ind", "nds", "nld", "nob", "pol", "por", "ron", "rus", "slk", "spa", 
 "swc", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", "cmn"]
WLANGS = ["ar", "be", "bn", "bg", "ca", "cs", "da", "de", "el", 
"en", "eo", "fa", "fi", "fr", "ha", "sr", "he", "hi", "hu", "io", 
"ia", "is", "it", "ja", "kab", "ko", "ckb", "la", "lt", "mr", "mk",
 "id", "nds", "nl", "no", "pl", "pt", "ro", "ru", "sk", "es", "sw",
  "sv", "tt", "tl", "tr", "uk", "vi", "yi", "zh"]
LANGS = ["ara", "bel", "ben", "bul", "cat", "ces", "dan", "deu", "ell", 
"eng", "epo", "fas", "fin", "fra", "hau", "hbs", "heb", "hin", "hun", 
"ido", "ina", "isl", "ita", "jpn", "kab", "kor", "kur", "lat", "lit", 
"mar", "mkd", "msa", "nds", "nld", "nor", "pol", "por", "ron", "rus", 
"slk", "spa", "swa", "swe", "tat", "tgl", "tur", "ukr", "vie", "yid", 
"zho"]
# WV = WikiDataSet("corpus/wikipedia/test", langs=WLANGS)
TV = TatoebaDataset("corpus/tatoeba", "tatoeba_test.txt", langs=TLANGS)

2085801-element TatoebaDataset:
 "Pardonu, permesu al mi indiki tri erarojn en la supra artikolo." => "epo"
                                                "Mi ne malgajnos!" => "epo"
                                   "Kiom for estas la flughaveno?" => "epo"
                                  "Ni ne povas dormi pro la bruo." => "epo"
                                  "Kie ekas la flughavenaj busoj?" => "epo"
                                    "Mi ŝatus havi tason da kafo." => "epo"
                                            "Tio tro multekostas!" => "epo"
                              "Mi veturigos vin al la flughaveno." => "epo"
                                                     "Kio mankas?" => "epo"
                                                "Ĝi estas novega." => "epo"
                                                                   ⋮
           "Každou chvíli může dojít k velmi silnému zemětřesení." => "ces"
                               "Tahle práce rozhodně není snadn

In [3]:
import LanguageFinder: LanguageFind
detector_lf(t) = LanguageFind(t, 0).lang
import LanguageDetect: detect
detector_ld(t) = first(detect(t)).language
import Languages: LanguageDetector, isocode
Languages_detector = LanguageDetector()
function detector_ls(t)
    l = Languages_detector(t) |> first
    if l == nothing
        return ""
    else
     return l |> isocode
    end
end

detector_ls (generic function with 1 method)

In [4]:
function bmk(detector, D, langs)
    tp = zeros(length(langs))
    total = zeros(length(langs))
    for (x, y) in D[200:1000:end]
        y = normcode(y)
        if y in langs
            ind = findfirst(isequal(y), langs)
            y_ = ""
            try
                y_ = normcode(detector(x))
            catch
                println("Error: $x")
            end
            if y_ == y
                tp[ind] += 1
            end
            total[ind] += 1
        end
    end
    tp ./ total
end


bmk (generic function with 1 method)

In [5]:
@time bmk_ls = bmk(detector_ls, TV, LANGS);
@time bmk_ld = bmk(detector_ld, TV, LANGS);
# @time bmk_lf = bmk(detector_lf, TV, LANGS);
;

Error: 空腹はメンタル的に良くない。
Error: 彼女は優しい心の持ち主です。
Error: 彼らは二人とも親切で正直です。
Error: 任せて！
Error: 私はロンドンまでバスで行った。
Error: 君が成功できるかは君の努力次第だ。
Error: 我々は彼の間違いを気の毒に思う。
Error: 一家はニューヨークに移ったが、新しい環境にすぐに順応した。
Error: しかし、アメリカではふつう、スポーツチームは腕で選ばれ、学業のコースは学力によって選ばれます。
Error: 「ねえ、数学の宿題できた？」「あ、やばい、完全に忘れてた」
Error: 毎日10ずつ新しい単語を覚えている。
Error: なぜ飛行機にはパラシュートではなくライフジャケットがあるのか。
Error: トムはフランス語は話せないけど、フランス語の漫画を読むのは好きなんだ。
Error: トムは彼には投票権がないと言う。
Error: 本当だと思うよ。


Error: שנשתמש ב-du או ב-Sie?
Error: זה נשגב מבינתך.
Error: أحتاج نصيحتك.


Error: - Ez őz? - Az.


  3.820704 seconds (2.59 M allocations: 176.687 MiB, 3.28% gc time, 73.07% compilation time)


  7.953754 seconds (45.28 M allocations: 1.561 GiB, 4.02% gc time, 22.64% compilation time)


In [9]:
using Printf
fmt(v) = v isa Real ? (v!=0 ? (@sprintf "%0.2f%%" v*100) : "-") : v
tb = ["Languages"; bmk_ls ;; "LanguageDetect"; bmk_ld]# ;; "LanguageFind"; bmk_lf]
hd = [""; LANGS]
markdown_table(Tables.table(permutedims(tb), header=hd), formatter=fmt)

|                | ara     | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng    | epo    | fas     | fin    | fra    | hau     | hbs    | heb     | hin     | hun    | ido | ina | isl | ita    | jpn     | kab | kor     | kur | lat | lit    | mar    | mkd    | msa    | nds | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat | tgl    | tur    | ukr    | vie     | yid     | zho     |
|----------------|---------|---------|---------|--------|---------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|---------|---------|--------|-----|-----|-----|--------|---------|-----|---------|-----|-----|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|--------|--------|--------|---------|---------|---------|
|      Languages |  75.00% | 100.00% | 100.00% | 60.00% |       - | 35.71% | 25.00% | 91.94% | 100.00% | 80.77% | 64.83% |  66.67% | 79.31% | 83.93% | 100.00% | 66.67% |  95.00% |  66.67% | 72.15% |   - |   - |   - | 60.23% |  63.83% |   - | 100.00% |   - |   - | 78.95% | 73.33% | 43.75% | 75.00% |   - | 55.56% | 50.00% | 83.33% | 78.57% | 66.67% | 50.25% |      - | 65.00% |      - | 50.00% |   - | 86.67% | 50.68% | 63.89% |  80.00% | 100.00% | 100.00% |
| LanguageDetect | 100.00% |       - | 100.00% | 60.00% | 100.00% | 78.57% | 41.67% | 78.23% | 100.00% | 75.27% |      - | 100.00% | 93.10% | 75.89% |       - | 33.33% | 100.00% | 100.00% | 89.87% |   - |   - |   - | 64.33% | 100.00% |   - | 100.00% |   - |   - | 84.21% | 80.00% | 68.75% | 75.00% |   - | 66.67% | 75.00% | 91.67% | 75.00% | 66.67% | 79.60% | 50.00% | 61.25% | 50.00% | 90.00% |   - | 93.33% | 91.78% | 86.11% | 100.00% |       - |  92.86% |


In [8]:
markdown_table(Tables.table(permutedims(tb), header=hd),String, formatter=fmt)|>print

|                | ara     | bel     | ben     | bul    | cat     | ces    | dan    | deu    | ell     | eng    | epo    | fas     | fin    | fra    | hau     | hbs    | heb     | hin     | hun    | ido | ina | isl | ita    | jpn     | kab | kor     | kur | lat | lit    | mar    | mkd    | msa    | nds | nld    | nor    | pol    | por    | ron    | rus    | slk    | spa    | swa    | swe    | tat | tgl    | tur    | ukr    | vie     | yid     | zho     |
|----------------|---------|---------|---------|--------|---------|--------|--------|--------|---------|--------|--------|---------|--------|--------|---------|--------|---------|---------|--------|-----|-----|-----|--------|---------|-----|---------|-----|-----|--------|--------|--------|--------|-----|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|-----|--------|--------|--------|---------|---------|---------|
|      Languages |  75.00% | 100.00% | 100.00% | 60.00% |       - | 35.71% | 25.00